# Mini-projeto 05 — Polars

Replicar agregações usando Polars (expressions + lazy) e comparar com Pandas.

> Entregável de portfólio — gerado em 2026-01-30.


## Objetivo

        Executar uma análise equivalente ao Pandas usando Polars e salvar um parquet em `outputs/`.

        ## Entregáveis (para portfólio)

        - [ ] `outputs/sales_polars.parquet`
- [ ] `reports/compare_pandas_polars.md`

        ## Como usar

        1- Rode este notebook (kernel com o `.venv` do repo)  
        2- Gere **assets** (imagens/HTML) e **reports** (markdown/json) dentro desta pasta  
        3- Faça commit dos arquivos gerados para evidenciar o resultado no GitHub

In [ ]:
# Setup: detectar raiz do repositório + paths padrão
from pathlib import Path
import pandas as pd
import numpy as np

def find_repo_root(start: Path | None = None) -> Path:
    p = (start or Path.cwd()).resolve()
    for _ in range(12):
        if (p / "requirements.txt").exists() and (p / "README.md").exists():
            return p
        p = p.parent
    return (start or Path.cwd()).resolve()

ROOT = find_repo_root()
DATA_SAMPLE = ROOT / "data" / "sample"
DATA_SOURCE = ROOT / "data" / "source" / "bases-dados-analytics-powerbi-ml"

# Pasta do projeto (onde salvar assets/outputs/reports)
PROJ = ROOT / "projects" / "05_polars"
ASSETS = PROJ / "assets"
OUTPUTS = PROJ / "outputs"
REPORTS = PROJ / "reports"
for d in (ASSETS, OUTPUTS, REPORTS):
    d.mkdir(parents=True, exist_ok=True)

# Dataset padrão (sempre disponível)
sales_path = DATA_SAMPLE / "sales.csv"
customers_path = DATA_SAMPLE / "customers.csv"
sales = pd.read_csv(sales_path, parse_dates=["date"])
customers = pd.read_csv(customers_path, parse_dates=["signup_date"])

# Dataset real (opcional): se você adicionou o submodule/clone em dados/source/
has_real = DATA_SOURCE.exists()

print("ROOT:", ROOT)
print("Dataset sample:", sales.shape, customers.shape)
print("Dataset real disponível?", has_real)

In [ ]:
import polars as pl

# Ler CSV sample com Polars
pl_sales = pl.read_csv(sales_path)

# Lazy + expressions
lazy = (
    pl_sales.lazy()
    .group_by(["region", "category"])
    .agg(pl.col("revenue").sum().alias("revenue_sum"))
    .sort("revenue_sum", descending=True)
)
res = lazy.collect()
display(res.head(10))

out_parquet = OUTPUTS / "sales_polars.parquet"
res.write_parquet(out_parquet)
print("Salvo:", out_parquet)

# Comparação rápida (Pandas vs Polars)
pd_res = (
    sales.groupby(["region", "category"], as_index=False)["revenue"].sum()
    .sort_values("revenue", ascending=False)
    .head(10)
)

md = []
md.append("# Comparação Pandas vs Polars — 05_polars\n")
md.append("## Top 10 (Pandas)\n")
md.append(pd_res.to_markdown(index=False))
md.append("\n## Top 10 (Polars)\n")
md.append(res.head(10).to_pandas().to_markdown(index=False))
out = REPORTS / "compare_pandas_polars.md"
out.write_text("\n".join(md), encoding="utf-8")
print("Salvo:", out)